# Generalized Linear Model's

What are GLMs? In short: a generalization of **linear regression**.

## Quick recap of linear regression

In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.api import abline_plot
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# create artificial data
f = lambda x: 0.2*x - 1
x = np.linspace(0, 10, 100)   # collect 100 data points
e = np.random.normal(loc=0, scale=0.3, size=(100))
y = f(x) + e
#fig, ax = plt.subplots()

plt.scatter(x, y)

In [ ]:
X = sm.add_constant(x)
model = sm.GLM(y, X, family=sm.families.Gaussian())
fit = model.fit()
print(fit.summary())

In [ ]:
plt.scatter(x, y)
abline_plot(model_results=fit, ax=plt.gca())
plt.gca().set_title('Linear Regression')
plt.gca().set_ylabel('Independent variable')
plt.gca().set_xlabel('Dependent variable');

In [ ]:
OLSfit = sm.OLS(y, sm.add_constant(x, prepend=True)).fit()
print(OLSfit.summary())

## Receptive fields of cortical area M1 (MI) neurons [Truccolo et al. 2004]
### Smooth pursuit task
<img src="http://jn.physiology.org/content/jn/113/2/487/F1.medium.gif", width=300>
<div style="text-align: right"> [Addou et al, J. Neurophysiology 2015] </div>


### Tuning curves/receptive fields extracted using GLMs:
<img src="VelocityTuningCurves.png", width=400>

## Spatio-temporal activity of a network of retinal neurons [Pillow et al. 2008]
<img src="RetinaConnectivity.png", width=400>

## What do the trigeminal ganglion (Vg) neurons code for? [Bush et al. 2016]
<img src="MouseWhiskerDeflection.png", width=400>

# Poisson Neuron Model

Assume a Poisson model:

$$p(y|\lambda)=\frac{(\lambda\Delta)^y}{y!}\exp(-\lambda\Delta)$$

* $\lambda\Delta$ is the expected number of spikes in time interval $\Delta$
* $\lambda$ is the (time dependent) intensity of the Poisson process

Linear model for expectation value $E(y)$:
$$E(y|\vec{\theta}) = f(\vec{\theta}\cdot\vec{x})$$

For Poisson distribution we have:
* $E(y|\vec{\theta})=\lambda(\vec{\theta})$
* $f(\cdot)=\exp(\cdot)$

$$\lambda(\vec{\theta}) = \exp(\vec{\theta}\cdot\vec{x})$$

How about other distributions? $\rightarrow$ <a href="https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function">Wikipedia article</a>

### Optimal parameters?
Likelihood to see a spike train $Y=\{y_k\}_{k=0,\ldots,T}$ given the parameters $\vec{\theta}$

$$p(Y|\vec{\theta})=\prod_{k=1}^T\frac{(\lambda_k(\vec{\theta})\Delta)^{y_k}}{y_k!}\exp(-\lambda_k(\vec{\theta})\Delta)$$

and the log likelihood $\mathcal{L}(\vec{\theta})\equiv\log p(Y|\vec{\theta})$:

$$\mathcal{L}(\vec{\theta})=\sum_k y_k\log\lambda_k(\vec{\theta}) + y_k\log\Delta -\log y_k! -\lambda_k(\vec{\theta})\Delta$$
collecting parameter independent terms in a constant $c$:
$$\mathcal{L}(\vec{\theta})=\sum_{k} y_k\log\lambda_k(\vec{\theta}) -\Delta\sum_k\lambda_k(\vec{\theta}) + c$$


### Gradient ascent

Iterative optimization procedure:
$$\vec{\theta}_{n+1} = \vec{\theta}_{n} - H^{-1}\nabla\mathcal{L}(\vec{\theta}_{n}) $$

With gradient $\nabla\mathcal{L}(\vec{\theta})=\sum_{k} y_k\vec{x}_{k} -\Delta\sum_k\vec{x}_k\lambda_k(\vec{\theta})$ and Hessian $H=-\Delta\sum_k\vec{x}_k\vec{x}_k^T\lambda_k(\vec{\theta})$

### Giving parameters a meaning

<img src="GLM_flowdiagram.png">

$$\lambda_k = \exp(\vec{h}\cdot\vec{y}_{k,\tau}+\vec{a}\cdot\vec{x}_{k,\tau} + a_0)\text{   with   } \vec{y}_{k,\tau}=\{y_i\}_{i=k-1,\ldots,k-\tau}$$

In [ ]:
# produce artificial data

import numpy as np
import statsmodels.api as sm
import pylab as pl

# function to compute the instantaneous firing rate lambda
def lam(a,i,N,k,taum,Y):
	s=0  # sum
	for mi in range (0,taum):
		for ji in range (0,N+1):
			s = s + a[i,ji,mi]*Y[ji,k-mi-1]
	return np.exp(s)

# function to produce the artificial spike pattern
def art_spikedata(a,N,kmax,taum):
	Y = np.zeros(shape=(N+1,kmax+taum))
	Y[:N,0:taum]=(np.random.poisson(0.1, N*taum).reshape(N,taum)) 
	Y[N,:] = 1    # weight of the a-zero contribution ([a[:,N,0])
	for k in range(taum,kmax+taum):
		for i in range(N):
			Y[i,k]=np.random.poisson(lam(a, i, N, k, taum, Y),1)
	return Y

# definition of the parameters
N=3
taum=20
kmax=50000

# define parameters for generating the artificial spike train.
# i) neuron 0 influences neuron 2 following a cosine kernel.
# ii) neuron 1 influences neuron 2 following a sine kernel.
# iii) self-interactions follow exponentially decaying cosine.
a = np.zeros(shape=(N,N+1,taum))
for m in range(taum):
	a[2,0,m] = 0.05 * np.cos((np.pi/2.*m)/taum)  # 0 -> 2
	a[2,1,m] = 0.1 * np.sin((2*np.pi*m)/taum)    # 1 -> 2
	for i in range(N):   # self-interaction
		a[i,i,m] = -0.2*(i+1)/N * np.cos(2*(np.pi*m)/taum) * np.exp(-2*m/taum) #-(0.01*taum-0.01*m)
a[:,N,0] = 0.1   # offset for all neurons

# Generation of the artificial spike train
Y = art_spikedata(a, N, kmax, taum)
print(Y.shape)

In [ ]:
#definition of the spike number in each sliding window (vecorized) of dimension (N+1)*taum for every k[0:kmax]. In order to establish back the contribution of a[N+1,0] (wich correspond to the mathematical alpha(i,0)) we set all spike count to zero apart from one every taum
X = np.zeros(shape=[kmax,(N+1)*taum])
for k in range(kmax):
	X[k,:] = np.reshape(Y[:,k:k+taum],(N+1)*taum)
	X[k,N*taum:(N+1)*taum-1] = 0

#formulation of GLM
alpha = np.zeros(shape=[N,(N+1)*taum])
for i in range(N):
	model = sm.GLM(Y[i,taum:],X,family=sm.families.Poisson())
	alpha[i,:] = model.fit().params

#reshaping of the result in order to make them easily comparable with the expected results
alphaReshape=alpha.reshape(N,N+1,taum) #create a tensor
aR=alphaReshape[:,:,::-1] #inverting the order of element in row
# put alpha_i,0 to the beginning of the array rather than the end, more convenient for plotting
a = a[:,[-1]+np.arange(N),:] # using advanced slicing to re-order columns
aR = aR[:,[-1]+np.arange(N),:] # using advanced slicing to re-order columns


In [ ]:
#plot of result and of the expected result
fig= pl.figure()
x=np.arange(taum)    #definition of axes x
for i in range(N):   #definition of axes y
	for j in range(N):
		pl.plot(x,aR[i,j,:], '*', c='r')
		pl.plot(x,a[i,j,:], c='b')
		pl.ylim((-0.2,0.2))
		pl.xlabel(r'time [a.u.]',fontsize=17)
		pl.ylabel(r'$\alpha_{%d,%d}$ [a.u.]' % (i+1,j),fontsize=17)
		pl.savefig('alpha%d_%d.png' % (i+1,j))
		pl.show()